In [1]:
# libraries
import itertools
import jedi
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_profiling
import pygments
import re
import seaborn as sns
from scipy import stats as st
import sklearn
from tqdm import tqdm
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
# display for notebooks
%matplotlib inline

In [3]:
# Versions
print("jedi version:", jedi.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("Pygments version:", pygments.__version__)
print("re version:", re.__version__)
print("Seaborn version:", sns.__version__)
print("sklearn version:", sklearn.__version__)

jedi version: 0.16.0
numpy version: 1.18.1
pandas version: 1.0.1
Pygments version: 2.5.2
re version: 2.2.1
Seaborn version: 0.10.0
sklearn version: 0.22.1


In [4]:
# loading the dataset
url = "https://raw.githubusercontent.com/dssg-pt/covid19pt-data/master/data.csv"

In [5]:
# creating a dataframe
covid_pt = pd.read_csv(url)

In [6]:
# checking the dataframe
covid_pt.head()
covid_pt.tail()

,data,data_dados,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,...,confirmados_f,confirmados_m,obitos_arsnorte,obitos_arscentro,obitos_arslvt,obitos_arsalentejo,obitos_arsalgarve,obitos_acores,obitos_madeira,obitos_estrangeiro
23,20/03/2020,20/03/2020 00:00,1020,506,106,361,2,29,3,1,...,506.0,514.0,1,2,2,0,1,0,0,0
24,21/03/2020,21/03/2020 00:00,1280,644,137,448,3,31,3,5,...,649.0,631.0,4,4,3,0,1,0,0,0
25,22/03/2020,22/03/2020 00:00,1600,825,180,534,5,35,4,7,...,821.0,779.0,5,4,4,0,1,0,0,0
26,23/03/2020,23/03/2020 00:00,2060,1007,238,737,5,42,11,9,...,1080.0,980.0,9,5,8,0,1,0,0,0
27,24/03/2020,24/03/2020 00:00,2362,1130,293,852,6,46,12,11,...,1229.0,1133.0,9,11,8,0,1,1,0,0


# 📔 Dicionário dos dados

Uma explicação do conteúdo em `data.csv`. 

📝 _ARS_: Administração Regional de Saúde 

| Nome da coluna        | Significado           | Possíveis valores  |
| ------------- |:-------------:| -----:|
| `data` | Data da publicação dos dados. | DD-MM-YYYY |
| `data_dados` | Data e hora da recolha dos dados apresentados (quando omitida nos relatórios, assume-se como sendo a data da publicação dos dados). **Geralmente, os dados são reportados até às 24h do dia anterior à `data` (equivalentes às 00h do dia de `data`, sendo este último o formato utilizado).** | DD-MM-YYYY HH:MM|
| `confirmados` | Casos confirmados      | Inteiro >= 0 |
| `confirmados_arsnorte` | Casos confirmados na ARS Norte      | Inteiro >= 0 |
| `confirmados_arscentro` | Casos confirmados na ARS Centro      | Inteiro >= 0 |
| `confirmados_arslvt` | Casos confirmados na ARS Lisboa e Vale do Tejo      | Inteiro >= 0 |
| `confirmados_alentejo` | Casos confirmados na ARS Alentejo     | Inteiro >= 0 |
| `confirmados_arsalgarve` | Casos confirmados na ARS Algarve    | Inteiro >= 0 |
| `confirmados_acores` | Casos confirmados na Região Autónoma dos Açores | Inteiro >= 0 |
| `confirmados_madeira` | Casos confirmados na Região Autónoma da Madeira  |  Inteiro >= 0 |
| `confirmados_estrangeiro` | Casos confirmados no estrangeiro | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_novos` | Número de novos casos confirmados comparativamente ao dia anterior. É uma coluna calculada a partir da diferença nos casos `confirmados` entre dias consecutivos. | Inteiro >= 0 |
| `recuperados` | Total de casos recuperados | Inteiro >= 0 |
| `obitos` | Total de óbitos | Inteiro >= 0 |
| `internados` | Número de pacientes COVID-19 internados | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `internados_uci` | Número de pacientes COVID-19 internados em Unidades de Cuidados Intensivos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `lab` | Número de casos suspeitos a aguardar resultados laboratoriais | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `suspeitos` | Total de casos suspeitos (tendo a [definição sido actualizada](https://www.dgs.pt/directrizes-da-dgs/orientacoes-e-circulares-informativas/orientacao-n-002a2020-de-25012020-atualizada-a-250220201.aspx) a 29/02/2020) desde 01/01/2020  | Inteiro >= 0 |
| `vigilancia` | Número de casos sob vigilância pelas autoridades de saúde | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `n_confirmados` | Número de casos cuja suspeita de infecção não se confirmou | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `cadeias_transmissao` | Número de cadeias de transmissão do SARS-CoV-2 activas | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `transmissao_importada` | Número de casos confirmados com transmissão por via de infectados de outros países | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_0_9_f` | Número de casos confirmados do sexo feminino na faixa etária 0-9 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_0_9_m` | Número de casos confirmados do sexo masculino na faixa etária 0-9 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_10_19_f` | Número de casos confirmados do sexo feminino na faixa etária 10-19 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_10_19_m` | Número de casos confirmados do sexo masculino na faixa etária 10-19 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_20_29_f` | Número de casos confirmados do sexo feminino na faixa etária 20-29 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_20_29_m` | Número de casos confirmados do sexo masculino na faixa etária 20-29 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_30_39_f` | Número de casos confirmados do sexo feminino na faixa etária 30-39 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_30_39_m` | Número de casos confirmados do sexo masculino na faixa etária 30-39 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_40_49_f` | Número de casos confirmados do sexo feminino na faixa etária 40-49 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_40_49_m` | Número de casos confirmados do sexo masculino na faixa etária 40-49 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_50_59_f` | Número de casos confirmados do sexo feminino na faixa etária 50-59 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_50_59_m` | Número de casos confirmados do sexo masculino na faixa etária 50-59 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_60_69_f` | Número de casos confirmados do sexo feminino na faixa etária 60-69 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_60_69_m` | Número de casos confirmados do sexo masculino na faixa etária 60-69 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_70_79_f` | Número de casos confirmados do sexo feminino na faixa etária 70-79 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_70_79_m` | Número de casos confirmados do sexo masculino na faixa etária 70-79 anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_80_plus_f` | Número de casos confirmados do sexo feminino na faixa etária 80+ anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_80_plus_m` | Número de casos confirmados do sexo masculino na faixa etária 80+ anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_80_plus_m` | Número de casos confirmados do sexo masculino na faixa etária 80+ anos | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `sintomas_tosse` | Percentagem de casos infetados que reportaram o sintoma de tosse | fracção entre [0, 1] ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `sintomas_febre` | Percentagem de casos infetados que reportaram o sintoma de febre | fracção entre [0, 1] ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `sintomas_dificuldade_respiratoria` | Percentagem de casos infetados que reportaram o sintoma de dificuldade respiratória | fracção entre [0, 1] ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `sintomas_cefaleia` | Percentagem de casos infetados que reportaram o sintoma de cefaleia ou dores de cabeça | fracção entre [0, 1] ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `sintomas_dores_musculares` | Percentagem de casos infetados que reportaram o sintoma de dores musculares | fracção entre [0, 1] ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `sintomas_fraqueza_generalizada` | Percentagem de casos infetados que reportaram o sintoma de fraqueza generalizada | fracção entre [0, 1] ou _vazio_ para os dias em que a DGS não reportava este indicador |
| `confirmados_f` | Número total de confirmados do sexo feminino | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava directamente este indicador |
| `confirmados_m` | Número total de confirmados do sexo masculino | Inteiro >= 0 ou _vazio_ para os dias em que a DGS não reportava directamente este indicador |
| `obitos_arsnorte` | Total de óbitos na ARS Norte      | Inteiro >= 0 |
| `obitos_arscentro` | Total de óbitos na ARS Centro      | Inteiro >= 0 |
| `obitos_arslvt` | Total de óbitos na ARS Lisboa e Vale do Tejo      | Inteiro >= 0 |
| `obitos_alentejo` | Total de óbitos na ARS Alentejo     | Inteiro >= 0 |
| `obitos_arsalgarve` | Total de óbitos na ARS Algarve    | Inteiro >= 0 |
| `obitos_acores` | Total de óbitos na Região Autónoma dos Açores | Inteiro >= 0 |
| `obitos_madeira` | Total de óbitos na Região Autónoma da Madeira  |  Inteiro >= 0 |
| `obitos_estrangeiro` | Total de óbitos no estrangeiro | Inteiro >= 0 |

> Definições exactas de alguns destes termos constam do glossário do [Plano Nacional de Preparação e Resposta à Doença por novo coronavírus (COVID-19)](https://covid19.min-saude.pt/wp-content/uploads/2020/03/Plano-de-Conting%C3%AAncia-Novo-Coronavirus_Covid-19.pdf) (página 65 em diante).

Uma outra métrica com potencial interesse científico, o número de casos com base na data de início de sintomas, é também reportada pela DGS. No entanto, é apenas disponibilizado um gráfico de eixos esparsos, o que pode introduzir erros de aproximação na transcrição e comprometer a factualidade dos dados. Por essa razão, esta informação é propositadamente excluída. 

In [7]:
# checking the datatypes
covid_pt.dtypes

data                                  object
data_dados                            object
confirmados                            int64
confirmados_arsnorte                   int64
confirmados_arscentro                  int64
confirmados_arslvt                     int64
confirmados_arsalentejo                int64
confirmados_arsalgarve                 int64
confirmados_acores                     int64
confirmados_madeira                    int64
confirmados_estrangeiro              float64
confirmados_novos                      int64
recuperados                            int64
obitos                                 int64
internados                           float64
internados_uci                       float64
lab                                  float64
suspeitos                              int64
vigilancia                           float64
n_confirmados                        float64
cadeias_transmissao                  float64
transmissao_importada                float64
confirmado

In [8]:
# checking the size of the df
covid_pt.shape

(28, 56)

In [9]:
#gathering descriptive statistics on the df
round(covid_pt.describe(), 2)

,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_estrangeiro,confirmados_novos,...,confirmados_f,confirmados_m,obitos_arsnorte,obitos_arscentro,obitos_arslvt,obitos_arsalentejo,obitos_arsalgarve,obitos_acores,obitos_madeira,obitos_estrangeiro
count,28.00,28.00,28.00,28.00,28.00,28.00,28.00,28.00,13.00,28.00,...,6.00,6.00,28.00,28.00,28.00,28.0,28.00,28.00,28.0,28.0
mean,405.57,197.86,44.18,146.07,0.89,10.43,1.50,1.25,6.92,84.36,...,779.17,738.67,1.00,0.96,1.04,0.0,0.18,0.04,0.0,0.0
std,664.16,325.30,78.63,235.60,1.77,14.30,3.09,2.95,3.50,124.59,...,327.91,281.26,2.55,2.41,2.22,0.0,0.39,0.19,0.0,0.0
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,...,390.00,395.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
25%,5.50,2.75,1.00,1.75,0.00,0.00,0.00,0.00,5.00,2.00,...,541.75,543.25,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
50%,50.00,31.50,2.50,13.50,0.00,2.50,0.00,0.00,9.00,13.50,...,735.00,705.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
75%,496.50,219.25,56.75,195.75,0.50,15.75,1.50,0.25,9.00,123.50,...,1015.25,929.75,0.00,0.00,1.00,0.0,0.00,0.00,0.0,0.0
max,2362.00,1130.00,293.00,852.00,6.00,46.00,12.00,11.00,11.00,460.00,...,1229.00,1133.00,9.00,11.00,8.00,0.0,1.00,1.00,0.0,0.0


In [10]:
# checking the columns that the df has
covid_pt.columns

Index(['data', 'data_dados', 'confirmados', 'confirmados_arsnorte',
       'confirmados_arscentro', 'confirmados_arslvt',
       'confirmados_arsalentejo', 'confirmados_arsalgarve',
       'confirmados_acores', 'confirmados_madeira', 'confirmados_estrangeiro',
       'confirmados_novos', 'recuperados', 'obitos', 'internados',
       'internados_uci', 'lab', 'suspeitos', 'vigilancia', 'n_confirmados',
       'cadeias_transmissao', 'transmissao_importada', 'confirmados_0_9_f',
       'confirmados_0_9_m', 'confirmados_10_19_f', 'confirmados_10_19_m',
       'confirmados_20_29_f', 'confirmados_20_29_m', 'confirmados_30_39_f',
       'confirmados_30_39_m', 'confirmados_40_49_f', 'confirmados_40_49_m',
       'confirmados_50_59_f', 'confirmados_50_59_m', 'confirmados_60_69_f',
       'confirmados_60_69_m', 'confirmados_70_79_f', 'confirmados_70_79_m',
       'confirmados_80_plus_f', 'confirmados_80_plus_m', 'sintomas_tosse',
       'sintomas_febre', 'sintomas_dificuldade_respiratoria',
     

In [11]:
# dropping all columns except "data", "data_dados", "confirmados"... (etc), since I'm not
# interested in these info at this moment, for the purpose of this study
covid19_pt = covid_pt.filter(['data', 'data_dados', 'confirmados', 'confirmados_novos', 
                           'recuperados', 'obitos', 'internados', 'internados_uci', 'lab',
                           'suspeitos', 'vigilancia', 'n_confirmados', 'cadeias_transmissao',
                           'transmissao_importada'])

In [12]:
#checking the new shape 
covid19_pt.shape

(28, 14)

In [13]:
# rechecking the types
covid19_pt.dtypes

data                      object
data_dados                object
confirmados                int64
confirmados_novos          int64
recuperados                int64
obitos                     int64
internados               float64
internados_uci           float64
lab                      float64
suspeitos                  int64
vigilancia               float64
n_confirmados            float64
cadeias_transmissao      float64
transmissao_importada    float64
dtype: object

In [14]:
# looking at the tail
covid19_pt.tail()

,data,data_dados,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada
23,20/03/2020,20/03/2020 00:00,1020,235,5,6,126.0,26.0,850.0,7732,9008.0,5862.0,24.0,95.0
24,21/03/2020,21/03/2020 00:00,1280,260,5,12,156.0,35.0,1059.0,9854,13155.0,7515.0,NaN,104.0
25,22/03/2020,22/03/2020 00:00,1600,320,5,14,169.0,41.0,1152.0,11779,12562.0,9027.0,NaN,114.0
26,23/03/2020,23/03/2020 00:00,2060,460,14,23,201.0,47.0,1402.0,13674,11842.0,10212.0,NaN,142.0
27,24/03/2020,24/03/2020 00:00,2362,302,22,30,203.0,48.0,1783.0,15474,11842.0,11329.0,NaN,142.0


In [15]:
# chaging the Date Series into datetime
covid19_pt[["data", "data_dados"]] = covid19_pt[["data", "data_dados"]].apply(pd.to_datetime)

In [16]:
# making sure that the conversion was correct
covid19_pt.dtypes

data                     datetime64[ns]
data_dados               datetime64[ns]
confirmados                       int64
confirmados_novos                 int64
recuperados                       int64
obitos                            int64
internados                      float64
internados_uci                  float64
lab                             float64
suspeitos                         int64
vigilancia                      float64
n_confirmados                   float64
cadeias_transmissao             float64
transmissao_importada           float64
dtype: object

In [17]:
# rechecking the descriptive stats
round(covid19_pt.describe(), 2)

,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada
count,28.00,28.00,28.00,28.00,20.00,11.00,19.00,28.00,21.00,10.00,15.00,22.00
mean,405.57,84.36,2.36,3.21,100.75,26.45,466.32,3039.64,5263.62,6001.50,12.20,46.82
std,664.16,124.59,4.87,7.52,65.45,14.11,524.74,4559.67,4481.72,3380.03,7.69,46.46
min,0.00,0.00,0.00,0.00,9.00,9.00,30.00,25.00,81.00,1746.00,4.00,2.00
25%,5.50,2.00,0.00,0.00,39.50,17.50,83.00,113.00,496.00,3462.75,6.00,6.25
50%,50.00,13.50,0.00,0.00,98.00,20.00,281.00,423.00,4923.00,5325.00,11.00,36.00
75%,496.50,123.50,3.00,1.00,143.25,38.00,669.00,4289.25,8091.00,8649.00,18.50,68.75
max,2362.00,460.00,22.00,30.00,206.00,48.00,1783.00,15474.00,13155.00,11329.00,24.00,142.00


In [18]:
# since "data" and "data_dados" have essentially the same information, I'm dropping 
# "data_dados"
covid19_pt.drop("data_dados", axis = 1, inplace = True)

In [19]:
# documentation says: n_confirmados == "Número de casos cuja suspeita de infecção não se 
# confirmou". So I have no interest in this Series for the purpose of this study either and
# I'm also dropping it
covid19_pt.drop("n_confirmados", axis = 1, inplace = True)

In [20]:
# for the purpose of this study I'm also dropping confirmados_novos, recuperados and obitos
covid19_pt.drop(["confirmados_novos", "recuperados", "obitos"], axis = 1, inplace = True)

In [21]:
# checking
covid19_pt

,data,confirmados,internados,internados_uci,lab,suspeitos,vigilancia,cadeias_transmissao,transmissao_importada
0,2020-02-26,0,NaN,NaN,NaN,25,NaN,NaN,NaN
1,2020-02-27,0,NaN,NaN,NaN,51,NaN,NaN,NaN
2,2020-02-28,0,NaN,NaN,NaN,59,NaN,NaN,NaN
3,2020-02-29,0,NaN,NaN,NaN,70,NaN,NaN,NaN
4,2020-01-03,0,NaN,NaN,NaN,85,NaN,NaN,NaN
5,2020-02-03,2,NaN,NaN,NaN,85,NaN,NaN,NaN
6,2020-03-03,4,NaN,NaN,NaN,101,NaN,NaN,2.0
7,2020-04-03,6,NaN,NaN,NaN,117,81.0,NaN,4.0
8,2020-05-03,9,9.0,NaN,NaN,147,213.0,NaN,5.0
9,2020-06-03,13,13.0,NaN,30.0,181,354.0,5.0,5.0


In [22]:
# calculating  'R0'(número básico de reprodução)
covid19_pt["factor_propagacao"] = 1 + round(covid19_pt["confirmados"].pct_change(), 2)

In [23]:
# calculating number of days it will take to double the amount of infected at that rate.
covid19_pt["dias_duplicar"] = round(np.log(2) / np.log(covid19_pt["factor_propagacao"]), 0)

In [27]:
# creating a reorder of the columns
reorded = ['data', 'confirmados', 'factor_propagacao', 'dias_duplicar',
           'internados', 'internados_uci', 'lab', 'suspeitos', 'vigilancia', 
           'cadeias_transmissao', 'transmissao_importada']

In [28]:
# reordering the data
covid19_pt = covid19_pt[reorded]

In [29]:
# checking
covid19_pt

,data,confirmados,factor_propagacao,dias_duplicar,internados,internados_uci,lab,suspeitos,vigilancia,cadeias_transmissao,transmissao_importada
0,2020-02-26,0,NaN,NaN,NaN,NaN,NaN,25,NaN,NaN,NaN
1,2020-02-27,0,NaN,NaN,NaN,NaN,NaN,51,NaN,NaN,NaN
2,2020-02-28,0,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN,NaN
3,2020-02-29,0,NaN,NaN,NaN,NaN,NaN,70,NaN,NaN,NaN
4,2020-01-03,0,NaN,NaN,NaN,NaN,NaN,85,NaN,NaN,NaN
5,2020-02-03,2,inf,0.0,NaN,NaN,NaN,85,NaN,NaN,NaN
6,2020-03-03,4,2.00,1.0,NaN,NaN,NaN,101,NaN,NaN,2.0
7,2020-04-03,6,1.50,2.0,NaN,NaN,NaN,117,81.0,NaN,4.0
8,2020-05-03,9,1.50,2.0,9.0,NaN,NaN,147,213.0,NaN,5.0
9,2020-06-03,13,1.44,2.0,13.0,NaN,30.0,181,354.0,5.0,5.0
